```{contents}
```
## Node

A **Node** is the fundamental execution unit in LangGraph.
It represents a **single step of computation** that reads from shared state, performs an operation (LLM call, tool execution, logic, human input, etc.), and returns **partial state updates** that drive the next step of the graph.

Formally, a node is:

> **`Node : State → Partial<State>`**

---

### **1. Role of Nodes in LangGraph**

LangGraph models workflows as **state machines**.
Nodes are the **state transition operators**.

```
State_t  ──Node──▶  State_(t+1)
```

Each node:

1. Receives the current state
2. Executes a computation
3. Emits updates that are merged into the global state

---

### **2. Node Execution Contract**

```python
def node_fn(state: State) -> dict:
    return { "field": new_value }
```

| Component    | Meaning                   |
| ------------ | ------------------------- |
| `state`      | Current global state      |
| Return value | Partial state update      |
| Reducer      | Merges updates into state |

Nodes **never mutate state directly** — they only **return updates**.

---

### **3. Adding Nodes to a Graph**

```python
builder.add_node("analyze", analyze_node)
```

A node is identified by a **unique name** and a **callable**.

---

### **4. Node Types in LangGraph**

| Node Type      | Purpose              |
| -------------- | -------------------- |
| LLM Node       | Model inference      |
| Tool Node      | External API / tool  |
| Function Node  | Pure Python logic    |
| Router Node    | Select next path     |
| Human Node     | Human input          |
| Approval Node  | Manual validation    |
| Subgraph Node  | Nested workflow      |
| Async Node     | Concurrent execution |
| Streaming Node | Token streaming      |
| Fallback Node  | Recovery path        |
| Retry Node     | Automatic retry      |

---

### **5. Example: LLM Node**

```python
def reason(state):
    response = llm.invoke(state["input"])
    return {"analysis": response}
```

---

### **6. Example: Router Node**

```python
def router(state):
    if state["confidence"] > 0.8:
        return "final"
    return "retry"
```

Router nodes return the **name of the next node**.

---

### **7. Node Composition & Flow**

Nodes are connected by **edges**:

```python
builder.add_edge("reason", "act")
```

Complex behavior emerges by composing simple nodes.

---

### **8. Node Variants & Execution Modes**

| Variant            | Capability              |
| ------------------ | ----------------------- |
| Sync Node          | Standard execution      |
| Async Node         | `async def` node        |
| Streaming Node     | Token streaming         |
| Batch Node         | Process multiple inputs |
| Human Node         | Pauses graph            |
| Interruptible Node | Supports halt/resume    |

---

### **9. Node Safety & Reliability Features**

| Feature    | Purpose                       |
| ---------- | ----------------------------- |
| Retry      | Recover from transient errors |
| Timeout    | Prevent hanging               |
| Checkpoint | Resume after failure          |
| Tracing    | Observability                 |
| Validation | Enforce state correctness     |

---

### **10. Production Example**

```python
from langgraph.graph import StateGraph, END

class State(TypedDict):
    query: str
    answer: str

def llm_node(state):
    return {"answer": llm.invoke(state["query"])}

builder = StateGraph(State)
builder.add_node("llm", llm_node)
builder.set_entry_point("llm")
builder.add_edge("llm", END)

graph = builder.compile()
```

---

### **11. Mental Model**

A LangGraph node behaves like a **transactional function** in a distributed system:

> Read state → Compute → Emit diff → Merge → Transition

This makes nodes **deterministic, testable, traceable, and fault-tolerant**.

---

### **12. Why Nodes Matter**

Without nodes, workflows are static chains.
With nodes, LangGraph supports:

* Iteration
* Conditional logic
* Autonomy
* Human control
* Recovery
* Scaling

Nodes are the **atomic building blocks** of intelligent systems.


### Demonstration

In [1]:
from typing import TypedDict

class State(TypedDict):
    question: str
    answer: str
    confidence: float


In [2]:
def reason_node(state: State):
    # Simulating LLM behavior
    answer = f"Answering: {state['question']}"
    confidence = state.get("confidence", 0.0) + 0.4
    
    return {
        "answer": answer,
        "confidence": confidence
    }


In [3]:
def router_node(state: State):
    if state["confidence"] >= 0.9:
        return "final"
    return "reason"

def final_node(state: State):
    return {"answer": state["answer"] + " ✔ Finalized"}



In [4]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("reason", reason_node)
builder.add_node("final", final_node)

builder.set_entry_point("reason")

builder.add_conditional_edges("reason", router_node, {
    "reason": "reason",
    "final": "final"
})

builder.add_edge("final", END)

graph = builder.compile()


In [5]:
result = graph.invoke({
    "question": "What is LangGraph?",
    "answer": "",
    "confidence": 0.0
})

print(result)


{'question': 'What is LangGraph?', 'answer': 'Answering: What is LangGraph? ✔ Finalized', 'confidence': 1.2000000000000002}
